<a href="https://colab.research.google.com/github/Avinashbabu7/SENTIMENTAL_ANALYSIS/blob/main/SENTIMENT_ANALYSIS_ON_SST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing libraries


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
lr=LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
de_gi= DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=2, min_samples_leaf=5)
km=KMeans(n_clusters=2)
rf = RandomForestClassifier()
sv=SVC(kernel='poly')
xgb_1=XGBClassifier()
gb_1=GradientBoostingClassifier()
gnb_1=GaussianNB()
smx=SGDClassifier()

Loading data set


In [ ]:

!pip install pytreebank 
import pytreebank
sst = pytreebank.load_sst()

obtaining  data-set as sentences





In [ ]:
import pandas as pd
data_train = [
                (
                    tree.to_lines()[0],
                    tree.label
                )
                for tree in sst['train']
            ]
data_test = [(
  tree.to_lines()[0],
  tree.label
)
for tree in sst['test']
]
x_train=pd.DataFrame(data_train,columns=['sentence','label'])
x_test=pd.DataFrame(data_test,columns=['sentence','label'])
x_train_sent=x_train[['sentence']]
print(x_train_sent)
y_train_ind=x_train['label']
print(y_train_ind)
x_test_sent=x_test[['sentence']]
y_test_ind=x_test['label']
print(x_test_sent)
print(y_test_ind)

                                               sentence
0     The Rock is destined to be the 21st Century 's...
1     The gorgeously elaborate continuation of `` Th...
2     Singer/composer Bryan Adams contributes a slew...
3     You 'd think by now America would have had eno...
4                  Yet the act is still charming here .
...                                                 ...
8539                                    A real snooze .
8540                                     No surprises .
8541  We 've seen the hippie-turned-yuppie plot befo...
8542  Her fans walked out muttering words like `` ho...
8543                                In this case zero .

[8544 rows x 1 columns]
0       3
1       4
2       3
3       2
4       3
       ..
8539    0
8540    1
8541    3
8542    0
8543    1
Name: label, Length: 8544, dtype: int64
                                               sentence
0                        Effective but too-tepid biopic
1     If you sometimes like to go to the 

preprocessing data set


In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(nb_words=2500, lower=True,split=' ')
tokenizer.fit_on_texts(x_train_sent['sentence'])
tokenizer.fit_on_texts(x_test_sent['sentence'])
# print(len(tokenizer.word_index))
X = tokenizer.texts_to_sequences(list(x_train_sent['sentence']))
x=tokenizer.texts_to_sequences(list(x_test_sent['sentence']))
x=pad_sequences(x)
X = pad_sequences(X)
#print(x.shape)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Vocabulary


In [ ]:
import pickle
# embeding_matrix=np.zeros((len(),300))
vocabulary=[]
for token,index in tokenizer.word_index.items():
  vocabulary.insert(index,token) 
#pickle.dump(vocabulary,open('/content/drive/My Drive/vocabulary.pickle', 'wb'))
# for wor in vocabulary:
# len(vocabulary)
# len(tokenizer.word_index.keys())

loading glove embeddings


In [ ]:
glove_embed_sst=pickle.load(open('/content/drive/My Drive/Glove_embed_sst.pkl', 'rb'))

embedding_matrix


In [ ]:
embedding_matrix=np.zeros((len(vocabulary),300))
for word,vec in glove_embed_sst.items():
  ind=vocabulary.index(word)
  embedding_matrix[ind]=vec
embedding_matrix.shape

(17087, 300)

LSTM/Bi_LSTM for random embeddings


In [ ]:
#this is for random embedded sentences using lstm and bi-lstm
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding,Bidirectional
from keras.layers import Dense
lstm_out = 300
batch_size = 32
model = Sequential()
model.add(Embedding(len(vocabulary), 300,input_length = X.shape[1]))
model.add(Masking(mask_value=0))

model.add(Bidirectional(LSTM(lstm_out, recurrent_dropout= 0.2, dropout= 0.2)))#this is for bilstm

#model.add(LSTM(lstm_out, recurrent_dropout= 0.2, dropout= 0.2)) #this is for lstm

# model.add(Dropout(0.2))
#model.add(Dense(5,activation='softmax'))
model.add(Dense(2,activation='softmax')) #for sst2
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 46, 300)           5126100   
_________________________________________________________________
masking_1 (Masking)          (None, 46, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1442400   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1202      
Total params: 6,569,702
Trainable params: 6,569,702
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
 print(X.shape)

(8544, 46)


In [ ]:
ohe = OneHotEncoder()
#Y = ohe.fit_transform(np.array(y_train_ind).reshape(-1,1)).toarray()
#y = ohe.fit_transform(np.array(y_test_ind).reshape(-1,1)).toarray()
Y = ohe.fit_transform(np.array(y_sst2_t).reshape(-1,1)).toarray()
y = ohe.fit_transform(np.array(y_sst2_tes).reshape(-1,1)).toarray()
model.fit(X,Y, batch_size =batch_size, epochs = 5)

Epoch 1/5
267/267 [==============================] - 278s 1s/step - loss: 0.6510 - accuracy: 0.6088
Epoch 2/5
267/267 [==============================] - 270s 1s/step - loss: 0.4328 - accuracy: 0.8084
Epoch 3/5
267/267 [==============================] - 270s 1s/step - loss: 0.3289 - accuracy: 0.8623
Epoch 4/5
267/267 [==============================] - 271s 1s/step - loss: 0.2498 - accuracy: 0.8940
Epoch 5/5
267/267 [==============================] - 271s 1s/step - loss: 0.1746 - accuracy: 0.9315


In [ ]:
score,evalu=model.evaluate(x,y,verbose=2,batch_size=batch_size)

70/70 - 14s - loss: 0.6627 - accuracy: 0.7425


In [ ]:
print(evalu)

0.7425339221954346


LSTM/BiLSTM for glove embeddings


In [ ]:
#this is for glove embedded sentences using lstm and bi-lstm
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding,Bidirectional
from keras.layers import Dense
lstm_out = 300
batch_size = 32
model = Sequential()
 
model.add(Embedding(len(vocabulary),300,trainable=False,weights=[embedding_matrix]))
model.add(Dropout(0.2))
model.add(Masking(mask_value=0))
model.add(LSTM(lstm_out, recurrent_dropout = 0.2, dropout = 0.2))
#model.add(Bidirectional(LSTM(lstm_out, recurrent_dropout = 0.2, dropout = 0.2)))#this is for bi-lstm
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 300)         5126100   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
masking_5 (Masking)          (None, None, 300)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 1505      
Total params: 5,848,805
Trainable params: 722,705
Non-trainable params: 5,126,100
_________________________________________________________________
None


BERT_EMBEDDINGS


In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
#sentences = ['This framework generates embeddings for each input sentence',  'Sentences are passed as a list of string.', 'The quick brown fox jumps over the lazy dog.']
# sentence_embeddings = model.encode(x_train_sent['sentence'])

In [ ]:

sentence_embeddings_test=model.encode(x_test_sent['sentence'])

In [ ]:
sentence_embeddings=model.encode(x_train_sent['sentence'])

In [ ]:
#already trained bert embeddings
import pickle
sentence_embeddings=pickle.load(open('/content/drive/MyDrive/embeding.pickle', 'rb'))
len(sentence_embeddings)
sentence_embeddings_test=pickle.load(open('/content/drive/MyDrive/embeding_test.pickle','rb'))
#print(sentence_embeddings_test[0].size)
#print(x_train_sent)

Neural Network_model for bert


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
# sc = StandardScaler()
# X = sc.fit_transform(X)
ohe = OneHotEncoder()

y_test_ind = ohe.fit_transform(np.array(y_test_ind).reshape(-1,1)).toarray()
y_train_ind = ohe.fit_transform(np.array(y_train_ind).reshape(-1,1)).toarray()
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)
model = Sequential()
model.add(Dense(128, input_dim=(768), activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(np.array(sentence_embeddings), y_train_ind, epochs=1, batch_size=32)
y_pred = model.predict(np.array(sentence_embeddings_test))
y_pred=np.array(y_pred)
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(y_test_ind)):
    test.append(np.argmax(y_test_ind[i]))
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

267/267 [==============================] - 14s 3ms/step - loss: 1.4396 - accuracy: 0.3718
Accuracy is: 45.294117647058826


In [ ]:
y_train_ind.size

42720

different ml classifiers trying on bert embeddings
->


In [ ]:
#lr.fit(np.array(sentence_embeddings),np.array(y_train_ind))
#y_pred=lr.predict(np.array(sentence_embeddings_test))
#lr.fit(np.array(sentence_embeddings),np.array(y_sst2_t))
#y_pred=lr.predict(np.array(sentence_embeddings_test))



# knn.fit(sentence_embeddings,y_train_ind)
#knn.fit(sentence_embeddings,y_sst2_t)
#y_pred=knn.predict(np.array(sentence_embeddings_test))


# gb_1.fit(sentence_embeddings,y_train_ind)
#gb_1.fit(sentence_embeddings,y_sst2_t)
#y_pred=gb_1.predict(np.array(sentence_embeddings_test))

# gnb_1.fit(sentence_embeddings,y_train_ind)
#gnb_1.fit(np.array(sentence_embeddings),np.array(y_sst2_t))
#y_pred=gnb_1.predict(np.array(sentence_embeddings_test))



rf.fit(np.array(sentence_embeddings),np.array(y_train_ind))
#rf.fit(np.array(sentence_embeddings),np.array(y_sst2_t))
y_pred=rf.predict(np.array(sentence_embeddings_test))

#smx.fit(sentence_embeddings,y_train_ind)
#smx.fit(sentence_embeddings,y_sst2_t)
#y_pred=smx.predict(np.array(sentence_embeddings_test))

#xgb_1.fit(np.array(sentence_embeddings),y_train_ind)
#xgb_1.fit(np.array(sentence_embeddings),np.array(y_sst2_t))
#y_pred=xgb_1.predict(np.array(sentence_embeddings_test))

 

# l=[(r,z) for r,z in zip(y_test_ind,y_pred)]
# print(tuple(x)).[0:10]
# print(l)
accuracy_score(y_test_ind,y_pred)
#accuracy_score(y_sst2_tes,y_pred)
# print(y_train_ind)

0.467420814479638

TEXTBLOB


In [ ]:
from textblob import TextBlob
y_pr=[]
for sentence in list(x_test_sent['sentence']):
  y_pr.append(TextBlob(sentence).sentiment.polarity)
y_pr
df=pd.DataFrame(y_pr,columns=['sent'])
#df['sent']=pd.cut(df['sent'],bins=5,labels=[0,1,2,3,4])
df['sent']=pd.cut(df['sent'],bins=2,labels=[0,1]) #this is for sst2 data set
accuracy_score(x_test['label'],df['sent'])

0.20452488687782805

Vader Lexicon


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader=SentimentIntensityAnalyzer()
y_pr=[]
for sentence in list(x_test_sent['sentence']):
  y_pr.append(vader.polarity_scores(sentence)['compound'])
y_pr
df=pd.DataFrame(y_pr,columns=['sent'])
df['sent']=pd.cut(df['sent'],bins=5,labels=[0,1,2,3,4])
#df['sent']=pd.cut(df['sent'],bins=2,labels=[0,1]) #this is for sst2 data set
accuracy_score(x_test['label'],df['sent'])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


0.3153846153846154


Converting SST-5 to SST-2

In [ ]:
def convert(n):
  if(n>2):
    n=1
  else:
    n=0
  return n

In [ ]:
y_train_ind=np.array(y_train_ind)
y_test_ind=np.array(y_test_ind)
y_sst2_t=[]
y_sst2_tes=[]
for i in y_test_ind:
  y_sst2_tes.append(convert(i))
for i in y_train_ind:
  y_sst2_t.append(convert(i))
y_sst2_t=np.array(y_sst2_t)
y_sst2_tes=np.array(y_sst2_tes)
y_sst2_t
y_sst2_t.shape

(8544,)

PY-Torch sentence embeddings


In [ ]:
# !pip install pytorch-pretrained-bert
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    # Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

In [ ]:
import torch
torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()